# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [26]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv


In [2]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [3]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year", infer_datetime_format=True,parse_dates=True,)

file_path2 = Path("Data/neighborhoods_coordinates.csv")
neighborhood_map_coordinates = pd.read_csv(file_path2)




- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [4]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units_per_yr = (
        sfo_data[["housing_units"]].groupby([sfo_data.index.year]).mean()
    )
    
    housing_units_per_yr_fig= px.bar(housing_units_per_yr,
                                 title='Housing Units in San Francisco from 2010 to 2016', 
                                 labels={'value':'Housing Units', 'year':"Year"},
                                 range_y=(367500, 387500))
    return housing_units_per_yr_fig
    


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    average_housing_costs = (
    sfo_data[["sale_price_sqr_foot", "gross_rent"]].groupby([sfo_data.index.year]).mean()
    )
    average_montly_rent_fig = average_housing_costs.hvplot.line(
        y=["gross_rent"],
        xlabel= ("Years"),
        ylabel= ("Monthly Rent"),
        label='Average Gross Rent by Year', line_color= "#2F4F4F")
    return average_montly_rent_fig


def average_sales_price():
    """Average Sales Price Per Year."""
    
    average_housing_costs = (
    sfo_data[["sale_price_sqr_foot", "gross_rent"]].groupby([sfo_data.index.year]).mean()
    )
    average_saleprice_prsqft_fig = average_housing_costs.hvplot.line(
        y=["sale_price_sqr_foot"],
        xlabel= ("Years"),
        ylabel= ("Price per SqFt"),
        label='Average Price per SqFt by Year',
        line_color= "#8B0000")
    return average_saleprice_prsqft_fig

    



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    prices_by_neighborhood =(
        sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
        .groupby([sfo_data.index.year, "neighborhood"])
        .mean()
        .reset_index()
    )
    avg_price_sqft_neighborhood_fig = prices_by_neighborhood.hvplot.line(
        y=["sale_price_sqr_foot"],
        x="year",
        xlabel= ("Years"),
        ylabel= ("Avg. Sale Price per SqFt"),
        groupby= 'neighborhood',
        color= ("#2F4F4F"),
        width=600,
    )
    avg_monthly_rent_neighborhood_fig = prices_by_neighborhood.hvplot.line(
        y=["gross_rent"],
        x="year",
        xlabel= ("Years"),
        ylabel= ("Avg. Gross Rent Year"),
        groupby= 'neighborhood',
        color= ("#8B0000"),
        width=500,
    )
    return avg_monthly_rent_neighborhood_fig + avg_price_sqft_neighborhood_fig

    
    
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    most_expensive =( 
        sfo_data.groupby(["neighborhood"]).mean())
    most_exp_sort= most_expensive.nlargest(10, 'sale_price_sqr_foot').reset_index()
    most_exp_plot = most_exp_sort.hvplot.bar(
        y="sale_price_sqr_foot",
        x=("neighborhood"),
        xlabel= ("Neighborhood"),
        ylabel= ("Avg. Sale Price per Sqr Ft"),
        rot=90,
        title= ("The Top 10 Most Expensive Neighborhoods"),
        color= ("#2F4F4F"),
        bar_width=0.5,
    )
    return most_exp_plot



def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    prices_by_neighborhood =(
    sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
    .groupby([sfo_data.index.year, "neighborhood"])
    .mean()
    .reset_index()
)
    table = prices_by_neighborhood.groupby(['year', 'neighborhood','sale_price_sqr_foot','gross_rent' ]).mean().reset_index()
    most_exp_sqrft_vs_rent_plot = table.hvplot.bar(
        y= ("sale_price_sqr_foot", "gross_rent"),
        x="year",
        groupby= 'neighborhood', 
        rot=90,
        height= 400,
        width=600,
        xlabel= ("Neighborhood"),
        ylabel= ("Avg. Cost/Rental income"),
        title= ("Cost to purchase versus rental income"),
    )
    return most_exp_sqrft_vs_rent_plot

    
    
#def parallel_coordinates():
 #   """Parallel Coordinates Plot."""

    # YOUR CODE HERE!



#def parallel_categories():
 #   """Parallel Categories Plot."""
    
    # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""
    most_expensive =( 
    sfo_data.groupby(["neighborhood"]).mean())
    m_neighborhood= most_expensive.reset_index()
    mean_neighborhood= m_neighborhood.drop(columns=["neighborhood"])
    avg_neighborhood_locations= pd.concat([neighborhood_map_coordinates, mean_neighborhood], axis=1)
    map_1 = px.scatter_mapbox(
        avg_neighborhood_locations,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        hover_name= "Neighborhood",
        color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
        mapbox_style= "carto-positron",
    )
    return map_1

#def sunburst():
  #  """Sunburst Plot."""
    
    # YOUR CODE HERE!


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [27]:
# Create a Title for the Dashboard


# Create a tab layout for the dashboard

avgg_row=pn.Row(average_price_by_neighborhood)

avg_row= pn.Row(average_gross_rent)
avg_row.append(average_sales_price)



scatter_column = pn.Column(
    "## Neighborhood Map", neighborhood_map
)

    # Put bar plots in row
row_of_bar = pn.Row(top_most_expensive_neighborhoods)
row_of_bar.append(most_expensive_neighborhoods_rent_sales)




plots_as_column = pn.Column(
    "# San Francisco Rental Prices Dashboard",
    #row_of_parallel,
    row_of_bar,
    scatter_column,
    housing_units_column,
)

housing_units_column = pn.Column(
    "##Housing Units Per Year", housing_units_per_year
)



# Create the dashboard
panel_dashboard = pn.Tabs(
    ("All Plots", plots_as_column),
    ("San Francisco Neighborhood Map", scatter_column),
    ("Most Expensive Neighborhoods", row_of_bar),
    ("Average Gross Rent and Sales Price Per Year", avg_row),
    ("Average Prices by Neighborhood", avgg_row)
   # ("Statistical Plots", row_of_parallel)
)
panel_dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] Column
                [0] Column()
                [1] Row
                    [0] HoloViews(Bars, name='interactive24153')
            [1] Column
                [0] Column()
                [1] Row
                    [0] Row(name='interactive24239')
                        [0] HoloViews(DynamicMap, name='interactive24239')
                        [1] Column
                            [0] WidgetBox
                                [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                            [1] VSpacer()
        [2] Column
            [0] Markdown(str)
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure, name='interactive24146', visible=False)
        [3] Column
            [0] Markdown(str)
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure, name='interactive22470')
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive24146', visible=False)
    [2] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive24153')
        [1] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive24239')
                    [0] HoloViews(DynamicMap, name='interactive24239')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
    [3] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive23977')
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive24061')
    [4] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive23913')
                    [0] HoloViews(Layout, name='interactive23913')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()

## Serve the Panel Dashboard

In [34]:
# Serve the# dashboard
app= panel_dashboard


AttributeError: 'Tabs' object has no attribute 'run_server'

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [55]:
#housing_units_per_year()


In [57]:
#average_gross_rent()

In [61]:
#average_sales_price()

In [60]:
#average_price_by_neighborhood()

In [63]:
#top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()